In [3]:
#load libraries
import pandas as pd
import numpy as np

# % makes it in window instead of popout
%matplotlib inline

#pandas display options
from pandas import set_option
set_option("display.max_rows", 10)

In [4]:
#This is a function that takes a multiparametric data set with the name of the sample "extract_-dilution" as the rows 
#and biological parameters and returns a cleaned cp2 DataFrame indexed by Prefraction and a Dataframe to inform the
#dilution profiles.
def readBioData(file = ''):
    #open a tab delimitted file as a pandas dataframe
    cp1 = pd.read_table(file, sep = '\t')
    
    #This for loop makes two new columns 'Prefraction'and 'Dilution'
    for i in range(0, len(cp1['Features'])):
        name = cp1.get_value(col = 'Features', index = i)
        cp1.set_value(col='Prefraction',index = i, value = str(name.split('_')[0]))
        cp1.set_value(col='Dilution',index = i, value = int(name.split('-')[2]))
    
    #Index cp1 parameters by 'Prefraction' column
    cp1.index = cp1.Prefraction
    cp2 = cp1.iloc[0:,1:-2] #Clean CP Data
    
    #creates a dataframe with information about the diluted CP biological data
    dilutions = cp1['Dilution'][cp1['Dilution']>5]
    
    return dilutions, cp2

In [5]:
#This function takes a Markers Table from Unifi as a CSV File and returns a cleaned dataframe with prefractions 
#as its indices and mz_rt baskets as the columns
def markersTable(file = ''):
    marks = pd.read_csv(file)
    
    #For loop takes the labels from UNIFI and provides a clean 'Prefraction' Column
    for i in range(0, len(marks['PrimaryId'])):
        name = marks.get_value(col = 'PrimaryId', index = i)
        if '_' in str(name):
            marks.set_value(col='Prefraction',index = i, value = str(name).split('_')[1])
        else:
            marks.set_value(col='Prefraction',index=i, value='NaN')
    
    #Redefine indices as 'Prefractions'
    marks.index = marks.Prefraction
    
    return marks.iloc[4:,9:-1] #Slice of the data that only contains ints and floats

In [6]:
#This is a function that takes a cp dataframe with columns of cp parameters and rows indexed by Prefraction and returns
#an nxn matrix. Note only computes each value once xi,yi::yi,xi.
def nbyn(cp2):
    #import necessary libraries
    from scipy.stats import pearsonr
    
    #make nxn DataFrame
    nxn = pd.DataFrame(index = cp2.index, columns = cp2.index, dtype=float)
    
    #compute pearson correlations for lower half of nxn and reflect the other to save time
    for xi in nxn.index:        #loops over all rows
        for yi in nxn.loc[xi:,xi:].columns: #starts at diagonal and computes down a column and across the row
            if xi == yi:
                cos = 1
            else:
                #cos = coscore(cp2.loc[xi].values,cp2.loc[yi].values) 
                cos = pearsonr(cp2.loc[xi].values,cp2.loc[yi].values)[0]
            nxn.set_value(index = xi, col = yi, value = cos)
            nxn.set_value(index = yi, col = xi, value = cos)
    return nxn

#takes in two arrays and computes the cosine score between the two
def coscore(x, y):
	if not len(x) == len(y):
		print ("lengths not same")
		return None
	magx = sum([xi**2 for xi in x]) ** (1.0/2.0)
	magy = sum([yi**2 for yi in y]) ** (1.0/2.0)
	
	dot = 0.0
	for xi, yi in zip(x, y):
		dot += xi * yi
	dot /= float(magx * magy)
	return dot

In [7]:
def scale(dilutionTable, markersTable):
    scaled2 = markersTable.copy()
    for i in dilutionTable.index:
        for column in markersTable.columns:
            markersTable.set_value(col=column, index=i, value=5*scaled2.get_value(col=column, index=i)/dilutionTable[i])

In [8]:
diltab, cp = readBioData('3STDEV_Dilutions_ALL.txt')

In [9]:
marks = markersTable('Preliminary_MarkerTable.csv')

In [10]:
nxn = nbyn(cp)

In [11]:
scale(diltab, marks)

In [12]:
def blankRemove(marks):
    keeper = []
    blank = []
    for c in marks.columns:
        if "Blank" in marks[c][marks[c]>0].index:
            blank.append(c)
        else:
            keeper.append(c)
    return marks.loc[cp.index][keeper]

In [13]:
blanked = blankRemove(marks)
blanked

,181.98852_0.041,158.00325_0.042,183.98785_0.042,180.99025_0.043,213.43665_0.043,167.01342_0.044,318.89495_0.045,280.09632_0.046,282.91003_0.046,449.81558_0.046,...,394.79276_4.480,192.92237_4.480,179.01474_4.480,249.98378_4.480,230.95819_4.481,492.81335_4.481,288.91871_4.481,370.24553_4.483,167.01323_4.483,495.81331_4.484
Prefraction,,,,,,,,,,,,,,,,,,,,,
RLPA-1010A,0,0,0,0,0,0.0,0,0,0,0,...,0.00,0.00,0,0.00,0,0,0,0,0,0
RLPA-1010B,0,0,0,0,0,0.0,0,0,0,0,...,0.00,0.00,0,0.00,0,0,0,0,0,0
RLPA-1010C,0,0,0,0,0,0.0,0,0,0,0,...,0.00,0.00,0,198.44,0,0,0,0,0,0
RLPA-1010D,0,0,0,0,0,0.0,0,0,0,0,...,0.00,0.00,0,0.00,0,0,0,0,0,0
RLPA-1010E,0,0,0,0,0,1752.3,0,0,0,0,...,143.91,0.00,0,0.00,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RLPA-2032B,0,0,0,0,0,0.0,0,0,0,0,...,0.00,0.00,0,0.00,0,0,0,0,0,0
RLPA-2032C,0,0,0,0,0,0.0,0,0,0,0,...,0.00,0.00,0,0.00,0,0,0,0,0,0
RLPA-2032D,0,0,0,0,0,0.0,0,0,0,0,...,0.00,405.53,0,0.00,0,0,0,0,0,0


In [14]:
def emptyRemove(marks):
    keeper = []
    empty = []
    for c in marks.columns:
        if  len(marks[c][marks[c]>0].index) == 0:
            empty.append(c)
        else:
            keeper.append(c)
    return marks.loc[cp.index][keeper]

In [15]:
clean = emptyRemove(blanked)

In [16]:
clean

,181.98852_0.041,158.00325_0.042,183.98785_0.042,180.99025_0.043,213.43665_0.043,167.01342_0.044,318.89495_0.045,280.09632_0.046,282.91003_0.046,449.81558_0.046,...,181.98901_4.479,279.09296_4.479,394.79276_4.480,192.92237_4.480,249.98378_4.480,492.81335_4.481,288.91871_4.481,370.24553_4.483,167.01323_4.483,495.81331_4.484
Prefraction,,,,,,,,,,,,,,,,,,,,,
RLPA-1010A,0,0,0,0,0,0.0,0,0,0,0,...,0,0,0.00,0.00,0.00,0,0,0,0,0
RLPA-1010B,0,0,0,0,0,0.0,0,0,0,0,...,0,0,0.00,0.00,0.00,0,0,0,0,0
RLPA-1010C,0,0,0,0,0,0.0,0,0,0,0,...,0,0,0.00,0.00,198.44,0,0,0,0,0
RLPA-1010D,0,0,0,0,0,0.0,0,0,0,0,...,0,0,0.00,0.00,0.00,0,0,0,0,0
RLPA-1010E,0,0,0,0,0,1752.3,0,0,0,0,...,0,0,143.91,0.00,0.00,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RLPA-2032B,0,0,0,0,0,0.0,0,0,0,0,...,0,0,0.00,0.00,0.00,0,0,0,0,0
RLPA-2032C,0,0,0,0,0,0.0,0,0,0,0,...,0,0,0.00,0.00,0.00,0,0,0,0,0
RLPA-2032D,0,0,0,0,0,0.0,0,0,0,0,...,0,0,0.00,405.53,0.00,0,0,0,0,0


In [25]:
#This is going to be a Feature class that describes the distribrution 
#and intensities of each adduct throughout the dataset
class Feature():
    def __init__(self, mz_rt, marks, cp, nxn, aIDn):
        self.label = 'aID' + str(aIDn)
        self.subtable = marks[mz_rt][marks[mz_rt]>0]
        self.runs = marks[mz_rt][marks[mz_rt]>0].index
        self.mz = mz_rt.split('_')[0]
        self.rt = mz_rt.split('_')[1]
        self.subNXN = nxn.loc[self.runs,self.runs]
        self.cScore = Feature.clusterScore(self.runs, nxn)
        self.syntheticFingerprint = cp.loc[self.runs].mean()
        #self.aScore = Feature.activityScore(self.syntheticFingerprint)
        self.aScore = (sum(cp.loc[self.runs].mean()**2))**(1/2)
        #self.synthWeighted 
    
    #function taxes one DataFrame and one array: one NXN of pearson correlations from all runs 
    #and an array of all the runs
    def clusterScore(runs, nxn):
        if len(runs)==0:
            return 0
        else:
            subNXN = nxn.loc[runs,runs]**3
            return float(subNXN.sum().sum()-len(runs))/(2*len(runs))
    
    #This function takes the cp fingerprint DataFrame and a subtable of runs and peak heights
    def syntheticFingerprint(subtable, cp):
        synth = cp.loc[runs]
    
    #This is a function that takes the synthetic fingerprint and computes the magnitude in order to 
    #estimate the overall activity
    def activityScore(syn):
        aScore = 0
        for i in syn:
            aScore += i**2
        return aScore**(1/2)

In [26]:
def analyze(marks, cp, nxn):
    adducts = []
    aIDn = 0
    for c in marks.columns:
        i = Feature(c, marks, cp, nxn, aIDn)
        i.label = pd.DataFrame({'mz': i.mz,
                               'rt': i.rt ,
                               'Cluster Score': i.cScore,
                               'Activity Score': i.aScore,
                                'Runs': [i.runs],
                               },index = [i.label])
        adducts.append(i.label)
        aIDn +=1
    adductsTable = pd.concat(adducts)
    return adductsTable

In [28]:
g = analyze(clean, cp, nxn)
g

,Activity Score,Cluster Score,Runs,mz,rt
aID0,2.360881,0.000000,"Index(['RLPA-2002F'], dtype='object', name='Pr...",181.98852,0.041
aID1,0.843552,0.180682,"Index(['RLPA-2011A', 'RLPA-2027A'], dtype='obj...",158.00325,0.042
aID2,0.907865,0.056910,"Index(['RLPA-1011A', 'RLPA-2004E', 'RLPA-2009E...",183.98785,0.042
aID3,0.767491,0.051527,"Index(['RLPA-2011F', 'RLPA-2019D', 'RLPA-2023B...",180.99025,0.043
aID4,1.066262,0.000000,"Index(['RLPA-2009D'], dtype='object', name='Pr...",213.43665,0.043
...,...,...,...,...,...
aID42296,1.295856,0.000000,"Index(['RLPA-2031F'], dtype='object', name='Pr...",492.81335,4.481
aID42297,1.499700,0.212400,"Index(['RLPA-1011E', 'RLPA-1012D', 'RLPA-1020D...",288.91871,4.481
aID42298,2.014174,0.000030,"Index(['RLPA-2007C', 'RLPA-2011C'], dtype='obj...",370.24553,4.483
aID42299,0.772899,0.319701,"Index(['RLPA-1020F', 'RLPA-2002A', 'RLPA-2005A...",167.01323,4.483


In [29]:
active = g.iloc[0:][g['Activity Score']>3]

In [30]:
active_clustered = active.iloc[0:][active['Cluster Score'] > 0.15]

In [115]:
active_clustered['Activity Score'].max()

6.0768635221414

In [31]:
active_clustered

,Activity Score,Cluster Score,Runs,mz,rt
aID301,3.065165,0.331456,"Index(['RLPA-1010C', 'RLPA-2008C', 'RLPA-2021B...",192.15985,0.178
aID359,3.219402,0.159418,"Index(['RLPA-1021C', 'RLPA-2002F'], dtype='obj...",230.13983,0.183
aID410,3.383279,0.286310,"Index(['RLPA-1020D', 'RLPA-2001D', 'RLPA-2004D...",238.96339,0.186
aID552,3.216399,0.787962,"Index(['RLPA-1014D', 'RLPA-1014E', 'RLPA-2001D...",295.87345,0.198
aID760,3.838029,0.201924,"Index(['RLPA-1014D', 'RLPA-2004D'], dtype='obj...",264.96831,0.382
...,...,...,...,...,...
aID42021,3.278105,0.400359,"Index(['RLPA-1012F', 'RLPA-1020D', 'RLPA-1021C...",255.41161,4.155
aID42046,5.729919,0.409831,"Index(['RLPA-1010E', 'RLPA-2011E'], dtype='obj...",534.74252,4.164
aID42058,3.467233,0.177548,"Index(['RLPA-1010F', 'RLPA-1014D'], dtype='obj...",315.88256,4.167
aID42060,3.428926,0.212834,"Index(['RLPA-2002D', 'RLPA-2004A'], dtype='obj...",538.74941,4.168


In [1]:
a = len(active_clustered[]['Runs'].index)
len(a)

SyntaxError: invalid syntax (<ipython-input-1-5f72ca7fc685>, line 1)

In [33]:
active_clustered_singled = active_clustered[len(active_clustered['Runs'].index) > 1.0]

KeyError: True

In [36]:
a,b,c = Feature('181.98852_0.041', clean, cp, nxn),Feature('449.81558_0.046', clean, cp, nxn),Feature('394.79276_4.480',clean, cp, nxn)

In [38]:
a.runs

Index(['RLPA-2002F'], dtype='object', name='Prefraction')

In [46]:
cdd = pd.DataFrame({'mz': b.mz,
                               'rt': c.rt ,
                               'Cluster Score': c.cScore,
                               'Activity Score': c.aScore,
                               }, index = [0])

In [47]:
frames = [add,bdd,cdd]

In [50]:
comb = pd.concat(frames, ignore_index=True)

In [51]:
comb

,Activity Score,Cluster Score,mz,rt
0,2.360881,0,181.98852,0.041
1,5.723054,0,449.81558,0.046
2,5.945873,0,449.81558,4.480
